In [68]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

In [69]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
df_nvidia_to_csv = yf.download("NVDA", start="2020-01-01", end="2025-08-13")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'
nvidia_path = 'datasets/nvidia.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)
df_nvidia_to_csv.to_csv(nvidia_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)
df_nvidia = pd.read_csv(nvidia_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_23480\126190450.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_23480\126190450.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_23480\126190450.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_23480\126190450.py:4: FutureWarning: YF.download() has changed argument auto_adjus

In [70]:
df_apple.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.53850555419922,72.59888386623527,71.29229630932706,71.54588227171874,135480400
3,2020-01-03,71.83329010009766,72.59405542623057,71.60868452797844,71.76566667933918,146322800
4,2020-01-06,72.40568542480469,72.4443284378041,70.70301958367426,70.9541954829699,118387200


In [71]:
df_microsoft.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,153.0423126220703,153.14712365624922,150.86035596005522,151.28912308572728,22622100
3,2020-01-03,151.1366424560547,152.40389764073367,150.60306438648712,150.8508074465806,21116200
4,2020-01-06,151.52731323242188,151.59401776506908,149.12619721864283,149.6693128510685,20813700


In [72]:
df_amazon.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000
3,2020-01-03,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000
4,2020-01-06,95.14399719238281,95.18450164794922,93.0,93.0,81236000


In [73]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

df_nvidia['Date'] = df_nvidia['Price']
df_nvidia = df_nvidia.drop(index=[0,1], columns='Price')

In [74]:
df_apple.head()

,Close,High,Low,Open,Volume,Date
2,72.53850555419922,72.59888386623527,71.29229630932706,71.54588227171874,135480400,2020-01-02
3,71.83329010009766,72.59405542623057,71.60868452797844,71.76566667933918,146322800,2020-01-03
4,72.40568542480469,72.4443284378041,70.70301958367426,70.9541954829699,118387200,2020-01-06
5,72.0651626586914,72.67135607636844,71.84538465436648,72.41535254834196,108872000,2020-01-07
6,73.22442626953125,73.52631792855652,71.76810120120939,71.76810120120939,132079200,2020-01-08


In [75]:
df_microsoft.head()

,Close,High,Low,Open,Volume,Date
2,153.0423126220703,153.14712365624922,150.86035596005522,151.28912308572728,22622100,2020-01-02
3,151.1366424560547,152.40389764073367,150.60306438648712,150.8508074465806,21116200,2020-01-03
4,151.52731323242188,151.59401776506908,149.12619721864283,149.6693128510685,20813700,2020-01-06
5,150.14572143554688,152.13711630992105,149.89799290182077,151.8036372690105,21634100,2020-01-07
6,152.53732299804688,153.21383324803998,150.498283797712,151.43204560824276,27746500,2020-01-08


In [76]:
df_amazon.head()

,Close,High,Low,Open,Volume,Date
2,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000,2020-01-02
3,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000,2020-01-03
4,95.14399719238281,95.18450164794922,93.0,93.0,81236000,2020-01-06
5,95.34300231933594,95.69450378417969,94.60199737548828,95.2249984741211,80898000,2020-01-07
6,94.59850311279297,95.55000305175781,94.3219985961914,94.9020004272461,70160000,2020-01-08


In [77]:
df_nvidia.head()

,Close,High,Low,Open,Volume,Date
2,5.971746444702148,5.971746444702148,5.892342616375051,5.942872368471369,237536000,2020-01-02
3,5.876162528991699,5.919971911233805,5.827125931377221,5.852017636623186,205384000,2020-01-03
4,5.90080451965332,5.906031734617808,5.756682464335461,5.782818539157897,262636000,2020-01-06
5,5.972244739532471,6.018045311441172,5.884128406356931,5.9291821653038275,314856000,2020-01-07
6,5.98344612121582,6.024766291068211,5.927937929074284,5.968013252371926,277108000,2020-01-08


In [78]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 2 to 1411
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1410 non-null   object
 1   High    1410 non-null   object
 2   Low     1410 non-null   object
 3   Open    1410 non-null   object
 4   Volume  1410 non-null   object
 5   Date    1410 non-null   object
dtypes: object(6)
memory usage: 66.2+ KB


In [79]:
df_apple.describe()

,Close,High,Low,Open,Volume,Date
count,1410,1410,1410,1410,1410,1410
unique,1394,1410,1410,1410,1408,1410
top,139.26515197753906,230.8000030517578,227.07000732421875,228.00999450683594,90956700,2025-08-12
freq,2,1,1,1,2,1


In [80]:
df_apple.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
Date      0
dtype: int64

In [81]:
#INITIAL
dfs = [df_apple, df_amazon, df_microsoft, df_nvidia]
window = 14
short_period = 12
long_period = 26
signal_line_period = 9
years = list(range(2020,2026))
stocks = ['AAPL', 'AMZN', 'MSFT', 'NVDA']
X_train, X_test, y_train, y_test = None, None, None, None

In [82]:
#CONVERTING TO CORRECT FEATURE TYPES
def convert_type(df):
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(int)
    df['Date'] = pd.to_datetime(df['Date'])

In [83]:
#FEATURE ENGINEERING
def engineer_features(df):
    df['Price Change'] = df['Close'].diff().fillna(0)

    df['Gain'] = df.loc[df['Price Change'] > 0, 'Price Change']
    df['Loss'] = -df.loc[df['Price Change'] < 0, 'Price Change']

    df['Gain'] = df['Gain'].fillna(0)
    df['Loss'] = df['Loss'].fillna(0)

    df['Average Gain'] = df['Gain'].rolling(window=window).mean().fillna(0)
    df['Average Loss'] = df['Loss'].rolling(window=window).mean().fillna(0)

    df['RS'] = df['Average Gain'] / df['Average Loss']

    df['RSI'] = 100 - (100 / (1 + df['RS']))

    df['Short EMA'] = df['Close'].ewm(span=short_period,adjust=False).mean()
    df['Long EMA'] = df['Close'].ewm(span=long_period, adjust=False).mean()

    df['MACD'] = df['Short EMA'] - df['Long EMA']

    df['Signal Line'] = df['MACD'].ewm(span=signal_line_period, adjust=False).mean()
    
    df['MACD Histogram'] = df['MACD'] - df['Signal Line']
    
    df['Year'] = df['Date'].dt.year
    
    return df

In [84]:
#DROPPING NULL AND UNIMPORTANT COLUMNS/ROWS
def drop_features_and_na(df):
    df = df.drop(columns=['Gain', 'Loss'])
    df = df.dropna()
    return df

In [85]:
#PRINTING SEABORN PLOTS
def print_plots(df, stock):
    for year in years:
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='RSI', kind='line')
        plt.title(f'RSI trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()
        
        sns.relplot(data=df.loc[df['Year'] == year], x='Date', y='MACD', kind='line')
        plt.title(f'MACD trend for year {year} Stock {stock}')
        plt.figure(figsize=(20,10))
        plt.show()

In [86]:
#MODEL TRAINING AND EVALUATION
def split_data(df):
    X = df.drop(columns=['Date', 'Close'])
    y = df['Close']

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
def create_pipeline():
    model = clone(Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LinearRegression())
        ]))
    return model

In [88]:
def evaluate(model):
    kfold = KFold(n_splits=6, random_state=30, shuffle=True)

    params = {'lr__n_jobs': [10,20,50]}
    cv = clone(GridSearchCV(estimator=model,param_grid=params,cv=kfold))

    cv.fit(X_train, y_train)
    
    return {"best_params": cv.best_params_, "best_score": cv.best_score_}, cv

In [89]:
#MAIN OUTPUT
models = []

for i,df in enumerate(dfs):
    X_train, X_test, y_train, y_test = None, None, None, None
    
    convert_type(df)
    df = engineer_features(df)
    df = drop_features_and_na(df)
    
    X_train, X_test, y_train, y_test = split_data(df)
    
    model = create_pipeline()
    
    evaluation, cv = evaluate(model)
    df = df.drop(columns='Close')
    
    print("Best parameter:", evaluation['best_params']['lr__n_jobs'])
    print("Best score:", evaluation['best_score'])
    
    models.append({'model':cv,'X_test': X_test,'y_test': y_test})
    
    print(f"{stocks[i]}:")
    print(df.head())
    print()
    
    # print_plots(df, stocks[i]) TODO: must implement merged stocks for good plotting visuals

Best parameter: 10
Best score: 0.9996759728693373
AAPL:
         High        Low       Open     Volume       Date  Price Change  \
15  77.281827  76.634574  76.941293  101832400 2020-01-22      0.272911   
16  77.177970  76.233653  76.781892  104472000 2020-01-23      0.369507   
17  78.088470  76.685278  77.344612  146537600 2020-01-24     -0.222198   
18  75.296586  73.632564  74.883600  161940000 2020-01-27     -2.260551   
19  76.897821  75.398026  75.497047  162234000 2020-01-28      2.110825   

    Average Gain  Average Loss        RS        RSI  Short EMA   Long EMA  \
15      0.508210      0.208906  2.432724  70.868613  75.349469  74.270923   
16      0.534604      0.208906  2.559064  71.902728  75.618516  74.480357   
17      0.534604      0.174405  3.065306  75.401603  75.811987  74.657818   
18      0.493718      0.335873  1.469957  59.513462  75.627916  74.654685   
19      0.644491      0.311550  2.068664  67.412530  75.796906  74.808141   

        MACD  Signal Line  MAC

In [90]:
for i in range(len(models)):
    model_ = models[i]['model']
    X_test_ = models[i]['X_test']
    y_test_ = models[i]['y_test']
    
    y_pred = model_.predict(X_test_)

    print(f"Predicted values for {stocks[i]}:")
    print(y_pred[0:10])

    print(f"Correct values for {stocks[i]}:")
    print(y_test_[0:10])

Predicted values for AAPL:
[133.85929512 226.10599328 229.25878089 190.12903146 173.25180192
 151.84382934 141.87648438 213.6559888  206.69585007 192.09896378]
Correct values for AAPL:
378     133.879745
1280    227.203445
1234    229.056870
994     191.558228
493     172.296021
687     151.331161
708     141.161697
1407    213.008255
1354    206.625504
1003    191.974670
Name: Close, dtype: float64
Predicted values for AMZN:
[172.70814521 236.81359192 198.19323473 144.5461563  169.73795238
 119.68284569 115.99680622 221.57695455 204.5917905  153.27894222]
Correct values for AMZN:
378     172.007996
1280    237.419998
1234    197.119995
994     145.889999
493     169.567505
687     118.540001
708     115.250000
1407    222.309998
1354    204.070007
1003    153.419998
Name: Close, dtype: float64
Predicted values for MSFT:
[261.75887811 411.78426392 415.09758769 364.14656418 329.47220546
 235.73517723 232.9547904  526.23136358 457.11193249 369.63712339]
Correct values for MSFT:
378     2